In [ ]:
import yfinance as yf
import matplotlib.pyplot as plt
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
import time

# Function to send email with the latest exchange rate
def send_email(stock_symbol, plot_filename, latest_price, email_recipient):
    # Set up email details
    sender_email = "meetpatel.test@gmail.com"  # Replace with your email address (the sender's email)
    recipient_email = email_recipient  # This is passed as a parameter (the recipient's email)
    subject = f"{stock_symbol} - Today's Exchange Rate Update"
    
    # Ensure that latest_price is a scalar
    latest_price = latest_price.item()  # .item() ensures that it is a scalar
    
    body = f"Here is the latest update for {stock_symbol}:\n\nLatest Exchange Rate: {latest_price:.2f} INR.\n\nSee the attached graph for details."

    # Set up email server (using Gmail SMTP here)
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = recipient_email
    msg['Subject'] = subject

    # Attach the body and the plot image
    msg.attach(MIMEText(body, 'plain'))
    with open(plot_filename, 'rb') as f:
        msg.attach(MIMEImage(f.read(), name=plot_filename))

    # Send the email
    try:
        with smtplib.SMTP('smtp.gmail.com', 587) as server:
            server.starttls()
            server.login(sender_email, "zqwp xsbb pzfn lzjg")  # Use your Gmail app password
            server.sendmail(sender_email, recipient_email, msg.as_string())
        print(f"Email sent to {recipient_email}")
    except Exception as e:
        print(f"Error sending email: {e}")

# Function to get the latest EUR/INR exchange rate and plot
def fetch_eur_inr_data():
    # Get EUR/INR data using yfinance
    data = yf.download('EURINR=X', period='1d', interval='1m')  # Fetch 1 minute interval for 1 day

    # Get the most recent exchange rate (closing price)
    latest_price = data['Close'].iloc[-1]

    # Plot the data
    plt.figure(figsize=(10, 6))
    plt.plot(data.index, data['Close'], label='EUR/INR Exchange Rate', color='green')
    plt.xlabel('Time')
    plt.ylabel('EUR/INR Rate')
    plt.title('EUR/INR Exchange Rate')
    plt.legend()

    # Save the plot as an image
    plot_filename = 'eur_inr_plot.png'
    plt.savefig(plot_filename)
    plt.close()

    return plot_filename, latest_price

# Function to get the daily update and send the email
def daily_update(stock_symbol, email_recipient):
    # Fetch EUR/INR data and plot
    plot_filename, latest_price = fetch_eur_inr_data()

    # Send the email with the latest exchange rate
    send_email(stock_symbol, plot_filename, latest_price, email_recipient)

# Example usage
email_recipient = "meetpatel.test@gmail.com"  # Recipient's email address
while True:
    # Run the daily update
    daily_update(stock_symbol="EUR/INR", email_recipient=email_recipient)
    
    # Sleep for a day (86400 seconds)
    time.sleep(86400)  # 24 hours
